In [3]:
from matplotlib import pyplot as plt
import Functions.GlobalUtils as g_utils
import Functions.DataUtils as d_utils
import Functions.VideoUtils as v_utils
import DATA_INFO as data_info
import Functions.LocalDB as db
import numpy as np
from Functions.CNN import CNN
from tensorflow import keras
import os

run_name = ''
folds = range(5)

class_num = 10
augment_ratio = 2

data_folder_name = 'T{}.C{}.V{}.A{}/'.format(1,class_num,20, augment_ratio)
labeled_data_info = db.get_db(data_info.Labeled.path + data_folder_name+'data_info.txt', d_utils.Labeled_data_info)
f_max, f_min = d_utils.get_data_max_min(data_info.Trimmed.path, 'flow', '{:02d}data.txt')

test_indexes = labeled_data_info[0].test_videos_indexes
folder = 'Predictions/'

_borders = labeled_data_info[0].classes_borders
_borders[0] = 0

models = ["DenseNet121", "EfficientNetB0", "InceptionV3"]
for model_name in models:
    if not os.path.isdir(f"{folder}/{model_name}"):
        os.mkdir(f"{folder}/{model_name}")
    for fold in folds:
        for i in test_indexes:
            data_X, data_Y, real_flow = d_utils.single_vid_Data_Labeling(i, class_num, borders = _borders, resize_image = (75,75), gray_scale=False)

            # model_name = "DenseNet121"
            _model_name = f"F{fold}_C10_{model_name}.h5"
            full_model_name = f'models/{model_name}/{_model_name}'
            model = keras.models.load_model(full_model_name, custom_objects={"f1_score": g_utils.f1_score()})

            raw_pred = model.predict(data_X)
            _predicted = list(np.argmax(raw_pred, axis=-1))
            
                        
            db.insert(f'{folder}/{model_name}/{_model_name}.predict{run_name}.txt', CNN.prediction_data(
                raw_pred    = [list(x) for x in raw_pred],
                predicted   = _predicted, 
                class_num   = class_num,
                truth_class = data_Y, 
                truth_flow  = real_flow, 
                borders     = _borders,
                vid_name    = data_info.Synced.FileNames.video_name.format(i) + '.mp4',
                model_name  = full_model_name,
                f_max       = f_max,
                f_min       = f_min))




11/11 [==============================] - 2s 64ms/step 336/ 336 - ETA :   0m  0s - 62 step/s - passed t


In [11]:
import Functions.VideoUtils as v_utils
import Functions.LocalDB as db
import DATA_INFO as data_info
import numpy as np
frame_tale = 1
augment_ratio = 2
batch_size = 32
stp_patience = 10
class_num = 10
epochs = 100
folds = 5

data_folder_name = 'T{}.C{}.V{}.A{}/'.format(1,class_num,20, augment_ratio)
# cnn = CNN(class_num=class_num, frame_tale=frame_tale, augment_ratio=augment_ratio, batch_size=batch_size, 
#             early_stopping_patience=stp_patience, resize_images=(75, 75))    
for fold in range(5):
    # d_utils.update_K_Fold_Dataset(data_info.Labeled.path + cnn.dataset_folder_name, fold)
    frames_info = db.get_db(data_info.Labeled.path + data_folder_name+'frames_info.txt', v_utils.frame_info)
    vids = [{"Folds" : {}, "val" : 0, "test" : 0, "train" : 0} for i in range(14)]
    for frame in frames_info:
        x = v_utils.frame_info.cast(frame)
        if(x.n_k_fold in list(vids[x.video_number]["Folds"].keys())):
            vids[x.video_number]["Folds"][x.n_k_fold] = vids[x.video_number]["Folds"][x.n_k_fold] + 1
        else:
            vids[x.video_number]["Folds"][x.n_k_fold] = 1
        vids[x.video_number][x.mode] = vids[x.video_number][x.mode] + 1

    print("\n")
    for vid in vids:
        print("{:3d}  {:3d}  {:3d}  {}".format(vid["test"], vid["val"], vid["train"], vid["Folds"]))
    
    print("\n\n")
    folds = [sum([v["Folds"][f] for v in vids if 0 in v["Folds"].keys()]) for f in range(folds)]
    print(folds)
    
    print([round(100*(f-np.mean(folds))/np.mean(folds), 2) for f in folds])
    break
    




  0   64  262  {0: 64, 1: 70, 4: 62, 2: 64, 3: 66}
382    0    0  {-1: 382}
  0   80  319  {2: 89, 1: 74, 0: 80, 4: 76, 3: 80}
  0   81  308  {4: 94, 3: 71, 0: 81, 1: 72, 2: 71}
  0   70  304  {0: 70, 2: 79, 4: 77, 3: 67, 1: 81}
  0   70  297  {3: 71, 1: 76, 4: 71, 0: 70, 2: 79}
  0   74  302  {1: 71, 3: 91, 0: 74, 4: 76, 2: 64}
360    0    0  {-1: 360}
  0   75  268  {3: 66, 1: 73, 0: 75, 2: 65, 4: 64}
  0   81  308  {1: 68, 3: 84, 2: 71, 4: 85, 0: 81}
  0   79  305  {0: 79, 1: 84, 2: 83, 4: 72, 3: 66}
  0   74  295  {2: 67, 3: 82, 0: 74, 1: 80, 4: 66}
  0   65  294  {0: 65, 1: 63, 3: 68, 2: 78, 4: 85}
336    0    0  {-1: 336}



[813, 812, 810, 812, 828]
[-0.25, -0.37, -0.61, -0.37, 1.6]


<h1>Get Prediction Run Time</h1>
<h3>Converting to Frozen Model</h3>

In [ ]:
from Functions.CNN import CNN
# Converting to Frozen Model

CNN.freeze_model('T1.C10.V20.A1.CONV.32.32.FULLY.32.32.h5', 'models/', 'frozen_models')

<h3>Predict Using Frozen Model</h3>

In [ ]:
import cv2
import numpy as np
from Functions.CNN import CNN
import Functions.GlobalUtils as g_utils
import Functions.DataUtils as d_utils
import Functions.LocalDB as db
import DATA_INFO as data_info
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf

# Loading Prediction Data
class_num = 10
augment_ratio = 1
data_folder_name = 'T{}.C{}.V{}.A{}/'.format(1,class_num,20, augment_ratio)
labeled_data_info = db.get_db(data_info.Labeled.path + data_folder_name+'data_info.txt', d_utils.Labeled_data_info)
test_indexes = labeled_data_info[0].test_videos_indexes
folder = 'Predictions/'
_borders = labeled_data_info[0].classes_borders
_borders[0] = 0
i = test_indexes[0]
data_X, data_Y, real_flow = d_utils.single_vid_Data_Labeling(i,1, class_num, borders = _borders)
data_X = np.reshape(data_X, (np.shape(data_X)[0], np.shape(data_X)[1],np.shape(data_X)[2], 1))

# Loading model
model_name = 'T1.C10.V20.A1.CONV.32.32.FULLY.32.32.pb'
sess=tf.compat.v1.InteractiveSession()
frozen_graph="./frozen_models/" + model_name
with tf.io.gfile.GFile(frozen_graph, "rb") as f:
      graph_def = tf.compat.v1.GraphDef()
      graph_def.ParseFromString(f.read())
sess.graph.as_default()
tf.import_graph_def(graph_def)
input_tensor = sess.graph.get_tensor_by_name("x:0") 
output_tensor = sess.graph.get_tensor_by_name("Identity:0") 
timer = g_utils.timer()
times
for i in range(len(data_X)):
      timer.start()
      probs = sess.run(output_tensor, {'x:0': np.array([data_X[i]])})
      _predicted = list(np.argmax(probs, axis=-1))
      timer.stop()
      times.append(timer.exact_passed_time)
sess.close()
print('\n\n')
print('Average Runtime For Each Frame : ',timer.to_labeled_time(np.mean(times)))


<h1>Post Proccess - Plot - Evaluate</h1>

In [4]:
import Functions.GlobalUtils as g_utils
import numpy as np
def postproccess(predicted, truth_flow, borders, f_min, f_max, xlim):
        borders_denorm = [f_min] + list(g_utils.denormalize(borders[1:-1], f_max, f_min)) + [f_max]
        data_denorm = g_utils.denormalize(truth_flow[int(xlim[0]*20):int(xlim[1]*20)], f_max, f_min)
        pred_flow = [(borders_denorm[f]+borders_denorm[f+1]) / 2 for f in predicted[int(xlim[0]*20):int(xlim[1]*20)]]
        pred_flow_pp = g_utils.apply_kernel(pred_flow, np.median, (7, 1))
        pred_flow_pp = g_utils.apply_kernel(pred_flow_pp, np.mean, (7, 1))
        pred_flow_pp = g_utils.apply_kernel(pred_flow_pp, np.mean, (7, 1))
        
        return pred_flow, pred_flow_pp, data_denorm, borders_denorm

def limitation(predicted, data_denorm, xlim, borders_denorm):
        target_data = predicted[int(xlim[0]*20):int(xlim[1]*20)]
        min_data = [borders_denorm[d] for d in range(len(borders_denorm) - 1) if borders_denorm[d] < min(data_denorm) < borders_denorm[d+1]][-1]
        mid = borders_denorm[borders_denorm.index(min_data)] + (borders_denorm[borders_denorm.index(min_data)+1] - borders_denorm[borders_denorm.index(min_data)])/2
        if mid < min(data_denorm): min_data = mid
        max_data = [borders_denorm[d+1] for d in range(len(borders_denorm) - 1) if borders_denorm[d] < max(data_denorm) < borders_denorm[d+1]][0]
        mid = borders_denorm[borders_denorm.index(max_data)] - (borders_denorm[borders_denorm.index(max_data)] - borders_denorm[borders_denorm.index(max_data)-1])/2
        if mid > max(data_denorm): max_data = mid
        min_pred = borders_denorm[min(target_data)]
        max_pred = borders_denorm[max(target_data) + 1]
        time = np.arange(start=xlim[0], stop=xlim[1], step=1/20)
        ylim = [min(min_data, min_pred), max(max_data, max_pred)]
        return time, ylim

def plot_table(dict, folds, test_videos_count):
        decimals = 3
        data = np.array(list(dict.values()))
        data = data.reshape(len(dict.keys()), len(folds), test_videos_count)
        mean = np.mean(data, axis=2)
        mean = np.round(mean, decimals=decimals)
        mean_overall = np.mean(mean, axis=1)
        mean_overall = np.round(mean_overall, decimals=decimals)
        # metric_names = ['F1 (%)', 'Corr (%)', 'R² (%)', 'MAE (%)', 'MSE']
        metric_names = list(dict.keys())
        print("{:<10}".format("Metric"), end='')
        print((" | {:<10}"*len(folds)).format(*(["Fold " + str(f) for f in folds])), end='')
        print(" | {:<10}".format("Avg"))

        print("{:<10}".format("----------"), end='')
        print((" | {:<10}"*len(folds)).format(*(["----------"]*len(folds))), end='')
        print(" | {:<10}".format("----------"))

        for i in range(data.shape[0]):
                print("{:<10}".format(metric_names[i]), end='')
                for f in range(len(folds)):
                        print(" | {:<10.{}}".format(mean[i][f], 8), end='')
                print(" | {:<10.{}}".format(mean_overall[i], 8))

In [5]:
with g_utils.Section_Separator('imports'):
    import Functions.GlobalUtils as g_utils
    import Functions.DataUtils as d_utils
    import DATA_INFO as data_info
    import Functions.LocalDB as db
    from Functions.CNN import CNN
    import numpy as np
    import sklearn.metrics as metrics


models = ["DenseNet121", "EfficientNetB0", "InceptionV3"]

for model_name in models:
    with g_utils.Section_Separator('initials'):
            folds = range(5)
            class_num = 10
            augment_ratio = 1
            folder = 'Predictions/'
            fps = 20
            f_max, f_min = d_utils.get_data_max_min(data_info.Trimmed.path, 'flow', '{:02d}data.txt')
            correlation_total = np.array([])
            maes_total = np.array([])
            r_square_total = np.array([])
            mse_total = np.array([])
            rmse_total = np.array([])
            f1_score_total = np.array([])
            
    _test_indexes = [1, 7, 13]
    _view_videos = [0,1,2]
    for j in range(3):
        test_indexes = [_test_indexes[j]]
        view_videos = [_view_videos[j]]
        with g_utils.Section_Separator('initials'):
            folds = range(5)
            class_num = 10
            augment_ratio = 1
            folder = 'Predictions/'
            fps = 20
            f_max, f_min = d_utils.get_data_max_min(data_info.Trimmed.path, 'flow', '{:02d}data.txt')
            correlation = np.array([])
            maes = np.array([])
            r_square = np.array([])
            mse = np.array([])
            rmse = np.array([])
            f1_score = np.array([])
        for fold in folds:
            _file_name = f'{folder}{model_name}/F{fold}_C10_{model_name}.h5.predict.txt'
            data = db.get_db(_file_name, CNN.prediction_data)
            pp_flow_list = []
            class_corrs = []
            class_maes = []
            for i in view_videos:
                xlim = [0, len(data[i].predicted)/20]
                pred_flow, pred_flow_pp, data_denorm, borders_denorm = postproccess(data[i].predicted, data[i].truth_flow, data[i].borders, f_min, f_max, xlim)
                time, ylim = limitation(data[i].predicted, data_denorm, xlim, borders_denorm)
                
                pp_flow_list.append(pred_flow_pp)
                
                # with g_utils.Section_Separator('plots'):
                #     p = 0
                #     with g_utils.Section_Separator('post process'):
                #         plt.figure(figsize=(12, 6))
                #         plt.plot(time, data_denorm, linewidth=1, label='Flow', c='tab:blue')
                #         plt.plot(time, pred_flow, linewidth=2, label='Prediction', c='tab:gray')
                #         plt.plot(time, pred_flow_pp, label='Post Processed Prediction', c='tab:red', linestyle='dashed')
                #         plt.title('Flow vs Time, Video Number {}, Final Post Processed Output'.format(test_indexes[i]))
                #         plt.yticks(borders_denorm) ;plt.xlim(xlim) ;plt.ylim(ylim) ;plt.grid() ;plt.legend()
                #         plt.xlabel('Time (s)') ;plt.ylabel('Flow (L/Min)') ;plt.tight_layout()
                #         # plt.savefig(folder + 'Pred_F{}_C{}_{:02d}_{}_pp_final.png'.format(fold,class_num,i,p), dpi=500)
                #         plt.show()
                #         p = p + 1
                #         pass
                #     with g_utils.Section_Separator('final'):
                #         plt.figure(figsize=(12, 6))
                #         plt.plot(time, data_denorm, linewidth=1, label='Flow', c='tab:blue')
                #         plt.plot(time, pred_flow_pp, label='Post Processed Prediction', c='tab:red')
                #         plt.title('Flow vs Time, Video Number {}, Final Flow Predictions'.format(test_indexes[i]))
                #         plt.yticks(borders_denorm) ;plt.xlim(xlim) ;plt.grid() ;plt.legend()
                #         plt.xlabel('Time (s)') ;plt.ylabel('Flow (L/Min)') ;plt.tight_layout()
                #         # plt.savefig(folder + 'Pred_C{}_{:02d}_{}_pp.png'.format(class_num,i,p), dpi=500)
                #         plt.show()
                #         p = p + 1
                #         pass          
                #     pass
                
                r_square = np.append(r_square, metrics.r2_score(data_denorm, pred_flow_pp))
                correlation = np.append(correlation, g_utils.corr_np(data_denorm, pred_flow_pp))
                maes = np.append(maes, g_utils.mae(data_denorm, pred_flow_pp, [f_max, f_min]))
                mse = np.append(mse, metrics.mean_squared_error(g_utils.normalize(data_denorm,target_range=[0.5,-0.5]), g_utils.normalize(pred_flow_pp,target_range=[0.5,-0.5])))
                rmse = np.append(rmse, metrics.mean_squared_error(g_utils.normalize(data_denorm,target_range=[0.5,-0.5]), g_utils.normalize(pred_flow_pp,target_range=[0.5,-0.5]),squared=False))
                f1_score = np.append(f1_score, metrics.f1_score(data[i].predicted, data[i].truth_class, average='macro'))
        
        
        correlation = correlation*100
        maes = maes*100
        r_square = r_square*100
        f1_score = f1_score*100
        
        r_square_total = np.array(list(r_square_total)+ list(r_square))
        correlation_total = np.array(list(correlation_total)+ list(correlation))
        maes_total = np.array(list(maes_total)+ list(maes))
        mse_total = np.array(list(mse_total)+ list(mse))
        rmse_total = np.array(list(rmse_total)+ list(rmse))
        f1_score_total = np.array(list(f1_score_total)+ list(f1_score))
        
        # print(f'\nPerson number {j+1}')        
        # plot_table({'F1 (%)' : f1_score, 'Corr (%)' : correlation, 'R² (%)' : r_square, 'MAE (%)' : maes, 'MSE' : mse, 'RMSE' : rmse}, folds, len(test_indexes))

    print(f'\nTotal {model_name} : ')
    plot_table({'F1 (%)' : f1_score_total, 'Corr (%)' : correlation_total, 'R² (%)' : r_square_total, 'MAE (%)' : maes_total, 'MSE' : mse_total, 'RMSE' : rmse_total}, folds, 3)



Person number 1


Exception: db not found! (path : Predictions/F0_C10_DenseNet121.h5.predict.txt)

<h1>Get Post Proccess Run Time</h1>

In [ ]:
import Functions.GlobalUtils as g_utils
import Functions.DataUtils as d_utils
import DATA_INFO as data_info
import Functions.LocalDB as db
from Functions.CNN import CNN
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

times = []
class_num = 10
frame_tale = 1
validation_ratio = 0.2
folder = 'Predictions/'
arch = 'CONV.64.64.64.64.64.64.FULLY.64.64.64.64.64.64'
_file_name = folder + 'T{}.C{}.V20.A5.{}.h5.predict.txt'.format(
    frame_tale, class_num, arch)

f_max, f_min = d_utils.get_data_max_min(
    data_info.Trimmed.path, 'flow', '{:02d}data.txt')
data = db.get_db(_file_name, CNN.prediction_data)
fps = 20
for i in [0, 1, 2]:
    borders_denorm = [f_min] + list(g_utils.denormalize(data[i].borders[1:-1], f_max, f_min)) + [f_max]
    
    
    timer = g_utils.timer()
    timer.start()
    pred_flow = [(borders_denorm[f]+borders_denorm[f+1]) /2 for f in data[i].predicted]
    time = np.arange(start=0, stop=len(pred_flow)/fps, step=1/fps)
    pred_flow_pp = g_utils.apply_kernel(pred_flow, np.median, (7, 1))
    pred_flow_pp = g_utils.apply_kernel(pred_flow_pp, np.mean, (7, 1))
    pred_flow_pp = g_utils.apply_kernel(pred_flow_pp, np.mean, (7, 1))
    pred_flow_pp = g_utils.apply_kernel(pred_flow_pp, np.mean, (7, 1))
    timer.stop()
    times.append(timer.exact_passed_time/len(pred_flow_pp))

print(times)
print('\n', 'Average Post Process Time : ', timer.to_labeled_time(np.mean(times)))
